In [1]:
import sys

from src.data_loader import load_kaggle_data
import os
os.chdir("/home/rana-helal/PycharmProjects/fake_news_classifier")

df_kaggle = load_kaggle_data()


In [11]:
!python -m spacy download en_core_web_sm


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
from tqdm import tqdm
tqdm.pandas()


In [3]:
from src.preprocessing import remove_duplicates_and_missing, clean_for_ml

# Apply text cleaning for ML models

df_cleaned = remove_duplicates_and_missing(df_kaggle)


In [4]:
from src.preprocessing import clean_for_dl

df_cleaned['clean_text'] = df_kaggle['text'].progress_apply(clean_for_dl)


100%|██████████| 44898/44898 [00:21<00:00, 2134.43it/s]


In [6]:
from src.utils import save_cleaned_data

save_cleaned_data(df_cleaned, "kaggle_clean_dl.csv")

 Saved cleaned data to data/processed/kaggle_clean_dl.csv
